# RAG Using Azure AI Agent Service & Semantic Kernel

This code snippet demonstrates how to create and manage an Azure AI agent for retrieval-augmented generation (RAG) using the `Azure AI Agent Service` and `Semantic Kernel`. The agent processes user queries based on the retrieved context and provides accurate responses accordingly.

## Initializing the Environment

SQLite Version Fix
If you encounter the error:
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```

Uncomment this code block at the start of your notebook:

In [1]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importing Packages
The following code imports the necessary packages:

In [2]:
import os

# Azure imports for project client and credentials
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FileSearchTool, FilePurpose, ToolSet
from azure.identity import InteractiveBrowserCredential

# Semantic Kernel imports
from semantic_kernel.agents.azure_ai import AzureAIAgent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

### Project Client Initialization

Initializes the `AIProjectClient` using an interactive browser credential and a connection string from environment variables. It then uploads a file to the project and creates a vector store for file search retrieval.

Note: 
- You can find the connection string in the project settings.
- The `document.md` file contains the travel documents that the AI agent will retrieve and augment with semantic search results.

In [3]:
project_client = AIProjectClient.from_connection_string(
    credential=InteractiveBrowserCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

uploaded_file = project_client.agents.upload_file_and_poll(
    file_path="document.md",
    purpose=FilePurpose.AGENTS,
)
print(f"Uploaded file, file ID: {uploaded_file.id}")

vector_store = project_client.agents.create_vector_store_and_poll(
    file_ids=[uploaded_file.id],
    name="vector_store"
)
print(f"Created vector store, vector store ID: {vector_store.id}")

Uploaded file, file ID: assistant-VanQY1QAZQgSU2JDdM6nLD
Created vector store, vector store ID: vs_LjASqHsFNuEAUPAKxRcVb6SH


### File Search Tool and Toolset Creation

This snippet creates a `FileSearchTool` using the vector store ID and adds it to a `ToolSet`. This toolset will be used by the AI agent for file search retrieval to provide relevant context.

In [4]:
file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])
toolset = ToolSet()
toolset.add(file_search_tool)

### Prompt Plugin Definition
This class defines a `PromptPlugin` with a kernel function to build an augmented prompt using the retrieved context. The function ensures that the AI agent only answers based on the provided context.

In [5]:
class PromptPlugin:
    @kernel_function(
        name="build_augmented_prompt",
        description="Build an augmented prompt using file search retrieval context."
    )
    def build_augmented_prompt(query: str, retrieval_context: str) -> str:
        return (
            f"Retrieved Context:\n{retrieval_context}\n\n"
            f"User Query: {query}\n\n"
            "IMPORTANT: Answer ONLY based on the above context. If the context does not provide enough information, respond with 'Insufficient context provided.'"
        )

### Query Processing Function

This asynchronous function processes a user query by creating a message, running the agent, and listing the responses. It prints the responses to the console.

In [6]:
async def process_query(project_client, agent, thread_id: str, user_query: str):
    print(f"\nProcessing query: {user_query}")
    message = project_client.agents.create_message(
        thread_id=thread_id,
        role=AuthorRole.USER,
        content=user_query
    )

    run = project_client.agents.create_and_process_run(
        thread_id=thread_id,
        assistant_id=agent.id,
    )

    messages = project_client.agents.list_messages(
        thread_id=thread_id,
    )

    print("Responses:")
    main_data = messages.get('data', [])
    if main_data:
        first_msg = main_data[0]
        for content in first_msg.get("content", []):
            if content.get("type") == "text":
                print("Text:", content["text"].get("value"))
    
    print("\n" + "="*60 + "\n")

### Main Function


In [7]:
async def main():
    async with AzureAIAgent.create_client(
        credential=InteractiveBrowserCredential(),
        conn_str=os.environ["PROJECT_CONNECTION_STRING"],
    ) as client:
        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = (
            "You are an AI assistant that provides accurate information based on the provided context. "
            "Only use information from the retrieved context to answer questions. "
            "If the context doesn't contain relevant information, clearly state that. "
            "Provide concise and accurate responses."
        )
        
        # Create agent definition.
        agent_definition = await client.agents.create_agent(
            model="gpt-4o-mini",
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            toolset=toolset,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Add the PromptPlugin for retrieval-augmented generation.
        agent.kernel.add_plugin(PromptPlugin(), plugin_name="promptPlugin")
        
        # Create a conversation thread.
        thread = await client.agents.create_thread()
        print(f"Created thread, thread ID: {thread.id}")
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is Neural Network?"  # No relevant context.
        ]
        
        try:
            for user_query in user_inputs:
                await process_query(project_client, agent, thread.id, user_query)
        finally:
            # Clean up resources.
            await client.agents.delete_thread(thread.id)
            await client.agents.delete_agent(agent.id)
            project_client.agents.delete_file(uploaded_file.id)
            project_client.agents.delete_vector_store(vector_store.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Created thread, thread ID: thread_AStMkmZVWuUpz2ELu5lvVKQm

Processing query: Can you explain Contoso's travel insurance coverage?
Responses:
Text: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†source】.



Processing query: What is Neural Network?
Responses:
Text: The context does not contain any information about neural networks. Therefore, I cannot provide an explanation based on the uploaded documents.



Cleaned up agent, thread, file, and vector store.
